<a href="https://colab.research.google.com/github/retuyu88/digitalentkominfo/blob/master/%5B27_1%5D_CNN_vs_ANN_Fashion_MNIST_ADF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://i.imgur.com/UjutVJd.jpg" align = "center">

---
# Dataset Fashion-MNIST


Jangan lupa, pastikan Colab ini berjalan dengan GPU


In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision=7)

---
## Load Dataset FMNIST





In [0]:
import tensorflow as tf

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

print('X_train.shape =',X_train.shape)
print('y_train.shape =',y_train.shape)
print('X_test.shape  =',X_test.shape)
print('y_test.shape  =',y_test.shape)

---
## Visualisasi Data FMNIST


In [0]:
fig, ax = plt.subplots(2,10,figsize=(15,4.5))
fig.subplots_adjust(hspace=0.1, wspace=0.1)
for j in range(0,2):
    for i in range(0, 10):
        ax[j,i].imshow(X_train[i+j*10], cmap='gray')
        ax[j,i].set_title(y_train[i+j*10])
        ax[j,i].axis('off')
plt.show()

---
## Bagi Data Validasi
Ambil 10,000 data terakhir dari data latih menjadi data Validasi di dalam matrix `X_val` dan `y_val`

In [0]:
X_val = X_train[-10000:,:]
y_val = y_train[-10000:]

X_train = X_train[:-10000, :]
y_train = y_train[:-10000]

print('X_val.shape   =',X_val.shape)
print('y_val.shape   =',y_val.shape)
print('X_train.shape =',X_train.shape)
print('y_train.shape =',y_train.shape)

---
## Normalizing Data

In [0]:
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

# mean_image = np.mean(X_train, axis = 0)
# X_train -= mean_image
# X_val -= mean_image
# X_test -= mean_image

X_train = np.expand_dims(X_train,-1)
X_val = np.expand_dims(X_val,-1)
X_test = np.expand_dims(X_test,-1)

print('X_train.shape =',X_train.shape)
print('X_val.shape   =',X_val.shape)
print('X_test.shape  =',X_test.shape)


---
## One Hot Matrix



 Ubah vektor target `y_train`, `y_val`, dan `y_test` menjadi bentuk One-Hot Matrix

In [0]:
from tensorflow.keras.utils import to_categorical

y_train_hot = to_categorical(y_train.ravel(), 10)
y_val_hot = to_categorical(y_val.ravel(), 10)
y_test_hot = to_categorical(y_test.ravel(), 10)

print('y_train_hot.shape =',y_train_hot.shape)
print('y_val_hot.shape   =',y_val_hot.shape)
print('y_test_hot.shape  =',y_test_hot.shape)

---

# Three-Layer Neural Network

Berikutnya, mari kita bangun Jaringan Saraf Tiruan sedalam 3 layer untuk mengklasifikasikan dataset FMNIST ini


---
## Import Modul



In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

num_data    = X_train.shape[0]
X_dim       = X_train.shape[1]
num_classes = y_train_hot.shape[1]

---
## Model Neural Network


In [0]:
# create model compact sequential

model_ann = Sequential([
  Flatten(input_shape=(28,28,1)),
  Dense(500, activation="relu"),
  Dense(200, activation="relu"),
  Dense(num_classes, activation="softmax")
])

model_ann.summary()

---
## Melatih Model

In [0]:
# Compile model
model_ann.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])


num_epochs = 15
batch_size = 1000

hist_ann = model_ann.fit(X_train, y_train_hot, 
          validation_data=(X_val, y_val_hot),
          epochs=num_epochs, batch_size=batch_size, 
          verbose=2)

---
## Evaluasi Model


In [0]:
scores_ann = model_ann.evaluate(X_test, y_test_hot, verbose=1)
print("\nModel Accuracy: %.2f%%" % (scores_ann[1]*100))

---

# Three-Layer Convolutional Neural Network
Sebagai perbandingan, mari kita bangun Convolutional Neural Network yang juga hanya 3 layer


---
## Import Modul



In [0]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

---
## Model Neural Network


In [0]:
# create model compact sequential

model_cnn = Sequential([
  Conv2D(32,(5,5),input_shape=(28,28,1), padding='same', activation="relu"),
  Conv2D(32,(5,5), padding='same', activation="relu"),
  MaxPooling2D(),
  Flatten(),
  Dense(num_classes, activation="softmax")
])

model_cnn.summary()

---
## Melatih Model

In [0]:
# Compile model
model_cnn.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])


num_epochs = 15
batch_size = 1000

hist_cnn = model_cnn.fit(X_train, y_train_hot, 
          validation_data=(X_val, y_val_hot),
          epochs=num_epochs, batch_size=batch_size, 
          verbose=2)

---
## Evaluasi Model


In [0]:
scores_cnn = model_cnn.evaluate(X_test, y_test_hot, verbose=1)
print("\nModel Accuracy: %.2f%%" % (scores_cnn[1]*100))

---
# Perbandingan ANN dan CNN

Sekarang, mari kita investigasi performa ANN dan CNN

---
## Jumlah Parameter

In [0]:
print('model ANN:')
model_ann.summary()

print('\n\nmodel CNN:')
model_cnn.summary()

**ANN memiliki parameter hingga 4x lebih banyak dari CNN**

In [0]:
print('total parameter ANN:', f'{ model_ann.count_params():,}')
print('total parameter CNN:', f'{ model_cnn.count_params():,}')

---
## Grafik Hasil Pelatihan

In [0]:
fig, ax = plt.subplots(1,3,figsize=(18,3))

ax[0].plot(hist_ann.history['loss'])
ax[0].plot(hist_cnn.history['loss'])
ax[0].set_title('Train Loss')
ax[0].set_ylabel('Loss')
ax[0].set_xlabel('Epoch')
ax[0].legend(['ANN', 'CNN'], loc='upper right')

ax[1].plot(hist_ann.history['acc'])
ax[1].plot(hist_cnn.history['acc'])
ax[1].set_title('Train Accuracy')
ax[1].set_ylabel('Accuracy')
ax[1].set_xlabel('Epoch')
ax[1].set_ylim(bottom=.7)
ax[1].legend(['ANN', 'CNN'], loc='lower right')

ax[2].plot(hist_ann.history['val_acc'])
ax[2].plot(hist_cnn.history['val_acc'])
ax[2].set_title('Validation Accuracy')
ax[2].set_ylabel('Accuracy')
ax[2].set_xlabel('Epoch')
ax[2].set_ylim(bottom=.7)
ax[2].legend(['ANN', 'CNN'], loc='lower right')
plt.show()

---
## Akurasi Data Uji

In [0]:
print("Akurasi ANN: %.2f%%" % (scores_ann[1]*100))
print("Akurasi CNN: %.2f%%" % (scores_cnn[1]*100))

---
## Pengecekan Overfitting

In [0]:
fig, ax = plt.subplots(1,2,figsize=(10,3))

ax[0].plot(hist_ann.history['acc'])
ax[0].plot(hist_ann.history['val_acc'])
ax[0].set_title('ANN Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].set_ylim(bottom=.7)
ax[0].legend(['train', 'val'], loc='lower right')

ax[1].plot(hist_cnn.history['acc'])
ax[1].plot(hist_cnn.history['val_acc'])
ax[1].set_title('CNN Accuracy')
ax[1].set_ylabel('Accuracy')
ax[1].set_xlabel('Epoch')
ax[1].set_ylim(bottom=.7)
ax[1].legend(['train', 'val'], loc='lower right')
plt.show()

Makin besar gap antara akurasi training dan validasi menandakan makin rentan terhadap overfitting

---
# Kesimpulan
* CNN memiliki parameter yang jauh lebih sedikit dari ANN, namun memiliki akurasi akhir yang lebih baik
* CNN Lebih tangguh terhadap *Overfitting*

<p>Copyright &copy; 2019 <a href=https://www.linkedin.com/in/andityaarifianto/>ADF</a> </p>